In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=20)
lngs = np.random.uniform(low=-180.000, high=180.000, size=20)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

In [6]:
cities

['rikitea',
 'scarborough',
 'vanderhoof',
 'chokurdakh',
 'ushuaia',
 'taian',
 'tanout',
 'krasnoselkup',
 'avarua',
 'hilo',
 'bluff',
 'langxiang',
 'busselton',
 'butaritari',
 'nikolskoye',
 'punta arenas',
 'qaanaaq',
 'arraial do cabo']

In [7]:
# Import the requests library.
import requests

# Import the API key.
from config import api_key

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key
# Create an endpoint URL for a city.
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=cf2ef14b68979549746bd13a432c79cf


In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                            "Lat": city_lat,
                              "Lng": city_lng,
                              "Max Temp": city_max_temp,
                              "Humidity": city_humidity,
                              "Cloudiness": city_clouds,
                              "Wind Speed": city_wind,
                              "Country": city_country,
                              "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | rikitea
Processing Record 2 of Set 1 | scarborough
Processing Record 3 of Set 1 | vanderhoof
Processing Record 4 of Set 1 | chokurdakh
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | taian
Processing Record 7 of Set 1 | tanout
Processing Record 8 of Set 1 | krasnoselkup
Processing Record 9 of Set 1 | avarua
Processing Record 10 of Set 1 | hilo
Processing Record 11 of Set 1 | bluff
Processing Record 12 of Set 1 | langxiang
Processing Record 13 of Set 1 | busselton
Processing Record 14 of Set 1 | butaritari
Processing Record 15 of Set 1 | nikolskoye
Processing Record 16 of Set 1 | punta arenas
Processing Record 17 of Set 1 | qaanaaq
Processing Record 18 of Set 1 | arraial do cabo
-----------------------------
Data Retrieval Complete      
-----------------------------


In [10]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,76.19,65,27,2.35,PF,2021-04-08 18:39:00
1,Scarborough,54.2797,-0.4044,48.00,67,100,5.99,GB,2021-04-08 18:37:00
2,Vanderhoof,54.0166,-124.0198,39.87,35,20,9.93,CA,2021-04-08 18:39:01
3,Chokurdakh,70.6333,147.9167,-12.06,95,100,4.76,RU,2021-04-08 18:39:01
4,Ushuaia,-54.8000,-68.3000,44.60,76,40,29.93,AR,2021-04-08 18:39:01
5,Taian,36.1853,117.1200,62.01,53,92,3.02,CN,2021-04-08 18:38:16
6,Tanout,14.9709,8.8879,89.62,6,0,10.98,NE,2021-04-08 18:39:02
7,Krasnoselkup,65.7000,82.4667,3.38,97,19,4.25,RU,2021-04-08 18:39:02
8,Avarua,-21.2078,-159.7750,77.00,94,75,5.75,CK,2021-04-08 18:39:03
9,Hilo,19.7297,-155.0900,73.40,68,75,9.22,US,2021-04-08 18:39:03


In [14]:
city_data_df.to_csv("../weather_database/weatherpy_database.csv")